<a href="https://colab.research.google.com/github/Sambhaji6529/Python_Assignment/blob/main/Assignment_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem 1: Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor.**

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df=pd.read_csv('/content/drive/MyDrive/imdb_data.csv')
df.head(2)

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435


In [3]:
df['profit'] = df.revenue - df.budget
df['profit'].replace([np.inf, -np.inf], 0, inplace=True) 
pd.options.display.float_format = '{:.4f}'.format
df.profit

0        -1685349
1        55149435
2         9792000
3        14800000
4         3923970
          ...    
2995      1596687
2996       180590
2997     24456761
2998    129963386
2999     47087155
Name: profit, Length: 3000, dtype: int64

In [4]:
df['new_cast'] = df.cast.fillna('[]').apply(ast.literal_eval)
df['new_genre'] = df.genres.fillna('[]').apply(ast.literal_eval)

df['new_cast'] = df.new_cast.apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
df['new_genre'] = df.new_genre.apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
df.head(2)

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,status,tagline,title,Keywords,cast,crew,revenue,profit,new_cast,new_genre
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.5754,...,Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,-1685349,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",[Comedy]
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.2489,...,Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,55149435,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...","[Comedy, Drama, Family, Romance]"


In [5]:
new_df = df.explode('new_cast')
top_actors = pd.DataFrame(new_df.new_cast.value_counts()).reset_index()
top_actors.columns = ['Actors','Number of movies']
top_actors.head()

,Actors,Number of movies
0,Robert De Niro,30
1,Samuel L. Jackson,30
2,Morgan Freeman,27
3,Bruce Willis,25
4,J.K. Simmons,25


In [6]:
#printing genre and mean 
grouped_actors = new_df.groupby('new_cast').agg({'new_genre':'sum','profit':'mean'}).reset_index()
grouped_actors.loc[(grouped_actors.new_cast=='Robert De Niro') | (grouped_actors.new_cast=='Samuel L. Jackson') | (grouped_actors.new_cast=='Morgan Freeman')].rename({'profit':'mean_profit'},axis=1)

,new_cast,new_genre,mean_profit
26518,Morgan Freeman,"[Action, Drama, Science Fiction, Thriller, Dra...",77610244.4815
30927,Robert De Niro,"[Drama, Comedy, Drama, Comedy, Crime, Romance,...",30043750.9333
32323,Samuel L. Jackson,"[Action, Adventure, Crime, Thriller, Crime, Dr...",226510055.6667


# **Problem 2 : Top 3 directors prefer which actors the most?**

In [7]:
# Your code here 
def find_directors(li):
  #print(li)
  job_iterator = 0

  if type(li)!= float:
    li = ast.literal_eval(li)
    while True:
      if li[job_iterator]['job']!='Director':
        job_iterator+=1
      else:
        return li[job_iterator]['name']

 
 
df['director_name'] = df.apply(lambda x: find_directors(x['crew']),axis = 1)
grouped_pop = df.groupby('director_name')['popularity'].mean().sort_values(ascending = False).reset_index()
grouped_pop.head(3)


,director_name,popularity
0,Tim Miller,187.8605
1,James Gunn,119.3113
2,Edgar Wright,118.9316


In [8]:
pop_dir_actors_1 = list(df[df['director_name'] == grouped_pop['director_name'][0]]['new_cast'])[0]
pop_dir_actors_2 = list(df[df['director_name'] == grouped_pop['director_name'][1]]['new_cast'])[0]
pop_dir_actors_3 = list(df[df['director_name'] == grouped_pop['director_name'][2]]['new_cast'])[0]

In [9]:
#actors prefered by tim miller
pop_dir_actors_1

['Ryan Reynolds',
 'Morena Baccarin',
 'Ed Skrein',
 'T.J. Miller',
 'Gina Carano',
 'Leslie Uggams',
 'Brianna Hildebrand',
 'Jed Rees',
 'Stefan Kapiƒçiƒá',
 'Randal Reeder',
 'Isaac C. Singleton Jr.',
 'Michael Benyaer',
 'Style Dayne',
 'Kyle Cassie',
 'Taylor Hickson',
 'Ayzee',
 'Naika Toussaint',
 'Justyn Shippelt',
 'Donna Yamamoto',
 'Hugh Scott',
 'Cindy Piper',
 'Emily Haine',
 'Aatash Amir',
 'Chad Riley',
 'Paul Belsito',
 'Darcey Johnson',
 'Kyle Rideout',
 'Jason William Day',
 'Stan Lee',
 'Benjamin Wilkinson',
 'Rachel Sheen',
 'Paul Lazenby',
 'Rob Hayter',
 'Heather Ashley Chase',
 'Fabiola Colmenero',
 'Victoria De Mare',
 'David Hardware',
 'Matthew Hoglie',
 'Tony Chris Kazoleas',
 'Greg LaSalle',
 'David Longworth',
 'Michael Neumeyer',
 'Sean Quan',
 'Anthony J. Sacco',
 'Olesia Shewchuk',
 'Dan Zachary']

In [10]:
#actors prefered by James Gunn
pop_dir_actors_2 

['Chris Pratt',
 'Zoe Saldana',
 'Dave Bautista',
 'Vin Diesel',
 'Bradley Cooper',
 'Lee Pace',
 'Michael Rooker',
 'Karen Gillan',
 'Djimon Hounsou',
 'John C. Reilly',
 'Glenn Close',
 'Benicio del Toro',
 'Wyatt Oleff',
 'Josh Brolin',
 'Sean Gunn',
 'Alexis Denisof',
 'Ophelia Lovibond',
 'Peter Serafinowicz',
 'Gregg Henry',
 'Laura Haddock',
 'Melia Kreiling',
 'Christopher Fairbank',
 'Mikaela Hoover',
 'Marama Corlett',
 'Emmett Scanlan',
 'Alexis Rodney',
 'Tom Proctor',
 'Spencer Wilding',
 'Fred',
 'James Gunn',
 'Stan Lee',
 'Stephen Blackehart',
 'Naomi Ryan',
 'Lloyd Kaufman',
 'Nathan Fillion',
 'Rob Zombie',
 'Tyler Bates',
 'Seth Green',
 'Krystian Godlewski',
 'Janis Ahern',
 'Solomon Mousley',
 'Lindsay Morton',
 'Robert Firth',
 'Nick Holmes',
 'Max Wrottesley',
 'Nicole Alexandra Shipley',
 'Sharif Atkins',
 'Brendan Fehr',
 'Ronan Summers',
 'Tomas Arana',
 'Dominic Grant',
 'Alison Lintott',
 'Keeley Forsyth',
 'Frank Gilhooley',
 'Enzo Cilenti',
 'Richard Katz'

In [11]:
#prefered by Edgar Wright	
pop_dir_actors_3 

['Simon Pegg',
 'Nick Frost',
 'Martin Freeman',
 'Eddie Marsan',
 'Paddy Considine',
 'Rosamund Pike',
 'Pierce Brosnan',
 'David Bradley',
 'Thomas Law',
 'Zachary Bailess',
 'Jasper Levine',
 'James Tarpey',
 'Luke Bromley',
 'Sophie Evans',
 'Samantha White',
 'Rose Reynolds',
 'Richard Hadfield',
 'Flora Slorach',
 'Francesca Reidie',
 'Charlotte Reidie',
 'Michael Smiley',
 'Alex Austin',
 'Jonathan Aris',
 'Jenny Bede',
 'Angie Wallis',
 'Paul Bentall',
 'Richard Graham',
 'Alice Lowe',
 'Rafe Spall',
 'Leo Thompson',
 'Steve Oram',
 'Julia Deakin',
 'Greg Townley',
 'Sebastian Zaniesienko',
 'Luke Scott',
 'Tyler Dobbs',
 'Samuel Mak',
 'Teddy Kempner',
 'Mark Kempner',
 'Nick Holder',
 'Darren Boyd',
 'Paul Kennington',
 'Mike Sarne',
 'Reece Shearsmith',
 'Nicholas Burns',
 'Mark Heap',
 'Kelly Franklin',
 'Stacey Franklin',
 'James Granstrom',
 'Gabe Cronnelly',
 'Patricia Franklin',
 'Bill Nighy',
 'Mark Donovan',
 'Ricky Champ',
 'Ken Bones']

# **Problem 3 : Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI.**

In [ ]:
movie producers directors

movie1 [p1, p2] [d1, d2]
movie2 [p3, p4] [d3, d4]

movie producers roi

movie1 p1 12
movie1 p2 13
movie2 p3 14
movie2 p4 15

In [ ]:
producer   movies            average_roi
p1.    [m1, m2, m3]    avg_roi(m1, m2, m3)
p2     [m2, m4, m5]    avg_roi(m2, m4, m5)

In [20]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,tagline,title,Keywords,cast,crew,revenue,profit,new_cast,new_genre,director_name
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.5754,...,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,-1685349,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",[Comedy],Steve Pink
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.2489,...,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,55149435,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...","[Comedy, Drama, Family, Romance]",Garry Marshall
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.3000,...,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,9792000,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",[Drama],Damien Chazelle
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.1749,...,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,14800000,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...","[Thriller, Drama]",Sujoy Ghosh
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.1481,...,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,3923970,"[Kim Kang-woo, Jo Jae-hyeon, Park Si-yeon, Kim...","[Action, Thriller]",Jong-seok Yoon


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [22]:
df_no_na_crew = df[~df['crew'].isna()]

In [23]:
len(df_no_na_crew)

2984

In [24]:
def get_producers_and_directors(string_rep_of_crew_list):
  try:
    crew_list = literal_eval(string_rep_of_crew_list)
    producer_list = []
    director_list = []
    for elem in crew_list:
      if elem['job']=='Producer':
        producer_list.append(elem['name'])
      if elem['job']=='Director':
        director_list.append(elem['name'])
    return producer_list, director_list
  except:
    return [], []

In [26]:
get_producers_and_directors(df_no_na_crew.loc[1, 'crew'])

([], [])